In [68]:
import os
import time
import re
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException


def scroll_shim(passed_in_driver, object):
    x = object.location['x']
    y = object.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % ( x, y)
    scroll_nav_out_of_way = 'window.scrollBy(0, -73);'
    passed_in_driver.execute_script(scroll_by_coord)
    passed_in_driver.execute_script(scroll_nav_out_of_way)
    
def feature_function(features):
    features_list=features.split("\n")
    long=len(features_list)
    count_till= 40  
    while long < count_till:
        features_list.append(float('nan'))
        long+=1
    return features_list



In [73]:
ean= ['8436028922536', '4242002972336']

In [74]:
def carrefour(ean_list):
    
    carrefour_info = pd.DataFrame()
    
    for ean in ean_list:

        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        browser = webdriver.Firefox()
        browser.get('https://www.carrefour.es/')

        time.sleep(3)

        action = ActionChains(browser)
        click_browser  = browser.find_element_by_xpath('//input[@id="atg_store_searchInput"]')
        action.move_to_element((click_browser)).click().perform()

        time.sleep(3)

        search = browser.find_element_by_xpath('//input[@class="ebx-search-box__input ebx-search-box__input-query"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)

        action = ActionChains(browser)
        click_image  = browser.find_element_by_xpath("(//img[@class='ebx-result-figure__img'])")
        action.move_to_element((click_image)).click().perform()

        time.sleep(3)

        title = browser.find_elements_by_xpath('//h1[@class="product-header__name"]')[0].text
        price = browser.find_elements_by_xpath('//span[@class="buybox__price--current"]')[0].text
        main_image = browser.find_element_by_xpath('//img[@class="main-image__image"]').get_attribute('src')

        action = ActionChains(browser)
        click_view_more  = browser.find_element_by_xpath("(//p[@class='product-details__show'])")
        if 'firefox' in browser.capabilities['browserName']:
            scroll_shim(browser, click_view_more)
        action.move_to_element((click_view_more)).click().perform()

        features = browser.find_elements_by_xpath('//div[@class="product-details__feature-container"]')[0].text
        features_list=feature_function(features)
        more_features = browser.find_elements_by_xpath('//div[@class="product-details"]')[0].text
        more_features_list=features.replace('\n',':').split(":")

        carrefour_data = pd.DataFrame({'ean.value': [ean],
                                                 'item_name.value': [title], 
                                                 'list_price.value_with_tax': [price[0]],
                                                 'short_product_description.value': features_list[0],
                                                 f'{features_list[1]}': [features_list[2]],
                                                 f'{features_list[3]}': [features_list[4]],
                                                 f'{features_list[5]}': [features_list[6]],
                                                 f'{features_list[7]}': [features_list[8]],
                                                 f'{features_list[9]}': [features_list[10]],
                                                 f'{features_list[11]}': [features_list[12]],
                                                 f'{features_list[13]}': [features_list[14]],
                                                 f'{features_list[15]}': [features_list[16]],
                                                 f'{features_list[17]}': [features_list[18]],
                                                 f'{features_list[19]}': [features_list[20]],
                                                 f'{features_list[21]}': [features_list[22]],
                                                 f'{features_list[23]}': [features_list[24]],
                                                 f'{features_list[25]}': [features_list[26]],
                                                 f'{features_list[27]}': [features_list[28]],
                                                 f'{features_list[29]}': [features_list[30]],
                                                 f'{features_list[31]}': [features_list[33]],
                                                 'more_features': [more_features_list],
                                                 'main_image': [main_image]
                                                })


        browser.close()
        carrefour_data = carrefour_data.drop(columns=['nan'])
        carrefour_data = carrefour_data.set_index('ean.value')
        carrefour_info = pd.concat([carrefour_info, carrefour_data], axis=1, sort=False)
    
    return carrefour_info
                


In [75]:
df = carrefour(ean_list)

In [76]:
df

,item_name.value,list_price.value_with_tax,short_product_description.value,more_features,main_image
ean.value,,,,,
8436028922536,Frigorífico Combi Icecool A+ CBNF1854,2,,[],https://static.carrefour.es/hd_510x_/crs/cdn_s...
